# Estudo do Atraso de Voos

## Obtenção e primeira exploração de dados

In [ ]:
import pandas as pd
from tqdm import tqdm

import database
from voos import voos_url

In [ ]:
microdados = pd.DataFrame()

urls = [ ]
for year in range( 2020, 2020 + 1 ):
    for month in range( 1, 12 + 1 ):
        month = str( month ) if month >= 10 else "0" + str( month )
        url = f"https://www.gov.br/anac/pt-br/assuntos/regulados/empresas-aereas/envio-de-informacoes/microdados/basica{year}-{month}.zip"
        urls.append( url )

for url in tqdm( urls ):
    parcial = pd.read_csv( url, encoding = "latin1", sep = ";" ).sample( frac = 0.05, random_state = 2 )
    microdados = pd.concat( [ microdados, parcial ], axis = 0 )

url_est = 'https://www.gov.br/anac/pt-br/assuntos/dados-e-estatisticas/dados-estatisticos/arquivos/resumo_anual_2010.csv'

Da definição:

Etapa Básica (flight stage): As etapas básicas são aquelas realizadas pela aeronave desde a sua decolagem até o próximo pouso, independentemente de onde tenha ocorrido o embarque ou o desembarque do objeto de transporte.

Os dados estatísticos das etapas básicas representam o status da aeronave em cada etapa do voo, apresentando a movimentação de cargas e passageiros entre os aeródromos de origem e destino da aeronave. É a operação de uma aeronave entre um a decolagem e o próximo pouso, a ligação entre dois aeródromos.

Etapa Combinada (On flight origin and destination - OFOD): As etapas combinadas identificam os pares de origem, onde ocorreu o embarque do objeto de transporte, e destino, onde ocorreu o desembarque do objeto de transporte, independentemente da existência de aeródromos intermediários, atendidos por determinado voo.

É a etapa de voo vista com foco no objeto de transporte (pessoas e/ou cargas), com base no embarque e no desembarque nos aeródromos relacionados. Os dados estatísticos da etapa combinada informam a origem e destino dos passageiros e cargas transportadas no voo, independentemente das escalas realizadas.


Aqui olharemos primeiramente para a etapa basica.

In [ ]:
microdados.duplicated( [ "id_basica" ] ).sum()

De Fato, id basica é uma pk ( identificador unico ) nessa tabela, e não temos nenhum registro duplicado!

In [ ]:
(microdados.isna().sum( axis = 0 ) / microdados.shape[ 0 ]).sort_values( ascending = False )

nr escala destino é 100% nulo. Doideira ( Isso provavelmente é algum tipo de informação que foi dropada ao longo do tempo. )

In [ ]:
unique = microdados.nm_empresa.value_counts()
cumulative = unique.cumsum()
cumulative = cumulative / unique.sum()
cumulative.sort_values( ascending = True ).iloc[ :15 ].plot( kind = "bar" )

Nenhuma surpresa aqui: a gigantesca maioria das etapas é realizada por poucas empresas
Aprox. 80% das etapas é feita por apenas 3;

In [ ]:
microdados.groupby( [ "nr_mes_referencia", "nm_mes_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

In [ ]:
microdados[ "nr_dia_semana_referencia" ] = pd.to_datetime( microdados[ "dt_referencia" ], infer_datetime_format = True ).dt.dayofweek

microdados.groupby( [ "nr_dia_semana_referencia", "nm_dia_semana_referencia" ] ).count()[ "id_basica" ].sort_index( ascending = True ).plot( kind = "bar" )

Aqui temos uma surpresa: Fim de semanas são dias com menos voos? Interessante. [quinta, sexta, segunda, quarta, terca ] são dias de ocupação bem proxima.

In [ ]:

microdados.query("nm_pais != 'BRASIL'").groupby("nm_pais").nunique()["nm_empresa"].nlargest(10).plot(kind="bar")

Essas são as nacionalidades das empresas não brasileiras dentro do dataset. USA domina, seguido de argentina, chile, venezuela

In [ ]:
microdados.groupby( [ "nm_municipio_destino", "nm_municipio_origem", "nm_empresa" ] ).nunique()

In [ ]:
microdados[ "nr_voo" ].value_counts()

O numero de voo mais comum é 2317 , seguido de 2316, seguido de 248, 15 e 247
Os menos comuns tem apenas 1 e são : 2263 , 6565, 595, 6566 , 593
Atente-se à amostragem

In [ ]:
microdados.query( "nr_voo == 2317" )[ [ "nm_empresa", "nr_voo", "nr_singular", "id_di", "nm_municipio_destino", "nm_municipio_origem", "dt_referencia" ] ]

In [ ]:
microdados[ "dt_referencia" ] = pd.to_datetime( microdados.dt_referencia, infer_datetime_format = True )

In [ ]:
microdados.groupby( [ "nr_ano_referencia", "nr_mes_referencia" ] ).mean()[ [ "nr_ask",
                                                                             "nr_rpk",
                                                                             "nr_atk",
                                                                             "nr_rtk",
                                                                             "kg_peso" ] ].sort_index( ascending = True ).plot()

## Análise dos Atrasos

In [ ]:
urls = list( voos_url )[ :5 ]


def download_voos( url ):
    return pd.read_csv( url, skiprows = 1, sep = ";" )


voos = pd.DataFrame()
for url in tqdm( urls ):
    parcial = download_voos( url )

    voos = pd.concat( (parcial, voos), axis = 0 )

In [ ]:
for data in [ "Chegada Prevista", "Chegada Real", "Partida Prevista", "Partida Real" ]:
    voos[ data ] = pd.to_datetime( voos[ data ], infer_datetime_format = True )

In [ ]:
voos[ "ano" ] = voos[ "Partida Real" ].dt.year
voos[ "mes" ] = voos[ "Partida Real" ].dt.month
voos[ "semana_ano" ] = voos[ "Partida Real" ].dt.week
voos[ "dia_semana" ] = voos[ "Partida Real" ].dt.dayofweek
voos[ "atraso_chegada" ] = (voos[ "Chegada Real" ] - voos[ "Chegada Prevista" ]).dt.seconds / 60
voos[ "atraso_saida" ] = (voos[ "Partida Real" ] - voos[ "Partida Prevista" ]).dt.seconds / 60
voos[ "atraso_ajustado" ] = voos[ "atraso_chegada" ] - voos[ "atraso_saida" ]
voos[ "atraso_chegada_abs" ] = abs( voos[ "atraso_chegada" ] )
voos[ "atraso_saida_abs" ] = abs( voos[ "atraso_saida" ] )
voos[ "atraso_ajustado_abs" ] = abs( voos[ "atraso_ajustado" ] )

In [ ]:
voos[ "atraso_chegada" ].plot( kind = "hist", bins = 100 )

Os dados concentrados à direita com cerca de 1400 minutos de atraso, o que corresponde a um dia, provavelmente são devido à mudança de um dia para outro.

In [ ]:
voos[ "duracao_real" ] = (voos[ "Chegada Real" ] - voos[ "Partida Real" ]).dt.seconds / 60
voos[ "duracao_prevista" ] = (voos[ "Chegada Prevista" ] - voos[ "Partida Prevista" ]).dt.seconds / 60

In [ ]:
(voos.query( "atraso_chegada > 0 & atraso_chegada < 240" ).groupby( [ "ano", "mes" ] )
 .agg(
    percentil_10 = ("atraso_chegada", lambda x: x.quantile( 0.10 )),
    percentil_20 = ("atraso_chegada", lambda x: x.quantile( 0.20 )),
    percentil_30 = ("atraso_chegada", lambda x: x.quantile( 0.30 )),
    percentil_40 = ("atraso_chegada", lambda x: x.quantile( 0.40 )),
    percentil_50 = ("atraso_chegada", lambda x: x.quantile( 0.50 )),
    percentil_60 = ("atraso_chegada", lambda x: x.quantile( 0.60 )),
    percentil_70 = ("atraso_chegada", lambda x: x.quantile( 0.70 )),
    percentil_80 = ("atraso_chegada", lambda x: x.quantile( 0.80 )),
    percentil_90 = ("atraso_chegada", lambda x: x.quantile( 0.90 )),
)[ [ "percentil_10",
     "percentil_20",
     "percentil_30",
     "percentil_40",
     "percentil_50",
     "percentil_60",
     "percentil_70",
     "percentil_80",
     "percentil_90" ] ].sort_index( ascending = True ).plot( figsize = (20, 20) )
 )

Doideira : um salto gigantesco no tamanho dos atrasos_saida e chegada, mas ambos acompanham uns aos outros.
Uma possivel explicação é a presença da olimpiada no mesmo ano /periodo em que vemos esse crescimento acentuado. No entanto, era de se esperar que passado esse evento, tudo se normalizasse, mas
não é o que observamos.

In [ ]:
voos.query( "atraso_chegada > 0" ).groupby( [ "ano", "mes" ] )[ [ "atraso_chegada",
                                                                  "atraso_saida" ] ].mean().sort_index( ascending = True ).plot()

Analisando somente os voos que atrasam, Vemos que a duração desses atrasos permanece

Será que os atrasos seguem algum tipo de padrão sazonal?

In [ ]:
voos.groupby( "semana_ano" ).mean()[ [ "atraso_chegada",
                                       "atraso_saida" ] ].plot()

Novamente os atrasos de aproximadamente 1400 minutos (~ 24 horas) aparecem. Optamos por tratar tais atrasos como uma mudança de dia.

In [ ]:
voos.query( "atraso_chegada > 0 " ).groupby( "dia_semana" ).median()[ [ "atraso_chegada",
                                                                        "atraso_saida" ] ].plot()

Parece haver um efeito sazonal do dia da semana no que diz respeito aos atrasos.
Peak atrasos ~ (terca )
Menor ~ ( quinta)

In [ ]:
voos

In [ ]:
ids_linhas = [ 'N', 'E', 'R' ]
ids_dis = [ '0', '4', 'C' ]

voos = voos.rename( {
    'Código Tipo Linha'      : 'id_tipo_linha',
    'Código Autorização (DI)': 'cd_di'
}, axis = 1 )
voos = voos.query( 'id_tipo_linha in @ids_linhas' )
voos = voos.query( 'cd_di in @ids_dis' )


Filtrando apenas pelos municípios de SP e RJ, vemos que existem algumas empresas que nem conhecemos, bem como aeroportos que não conhecemos.

Um filtro que faremos daqui em diante vai ser filtrar apenas voos de passageiros.
Isso signfica apenas os códigos ( N , I, E , R).

Um segundo filtro diz respeito aos voos internacionais: Não consideramos eles.
Isso inviabiliza os voos com código I.

Portanto consideramos apenas os códigos ( N , E , R )

Outra dimensão a ser considerada é o Digito Identificador.
Ele diz respeito ao tipo de voo realizado. Podem ser dos seguintes tipos:
Regular, Improdutivo, Não regular.
Nesta analise consideraremos apenas os voos regulares.
Estes são sinalizados com cd_di pertencente à ( 0 , 4 , C)

Outro ponto a ser considerado é a situação do voo.
Consideraremos apenas os voos Realizados

Correlação de atrasos com outras features
Algum aeroporto atrasa mais que os outros? Isso pode ser causado pelo clima do aeroporto ou pela movimentação daquele aeroporto
Horario do dia. ( Período manhã / tarde / noite )
Sera que algum voo atrasa mais?
Alguma companhia aerea?

Lotação?

## Após a análise dos dados, definimos as features e fizemos o merge com os dados de clima para obtenção do dataset final.

In [ ]:
voos = pd.read_sql( '''
with
	resumo_voos as (
	select
		date(dt_partida_prevista)  as dt_referencia
	  , dt_chegada_real
	  , dt_chegada_prevista
	  , sg_empresa_icao
	  , origem.nm_municipio  as municipio_origem
	  , destino.nm_municipio as municipio_destino
	  , timestampdiff( minute ,    dt_chegada_prevista , dt_chegada_real ) as atrasado
		from
			voos_sp_sj               voos
			inner join nrm_empresa   empresa on empresa.id_empresa = voos.id_empresa
			inner join nrm_aeroporto origem on origem.id_aerodromo = voos.id_aerodromo_origem
			inner join nrm_aeroporto destino on destino.id_aerodromo = voos.id_aerodromo_destino
		where
				dt_partida_prevista >= '2014-12-30 00:00:00'
		order by
			dt_partida_prevista
	)

SELECT *
	FROM
		resumo_voos
''', database.engine )
clima = pd.read_sql( """
select
	nm_municipio
  , dt_referencia + interval 1 day as dt_referencia
  , precipitacao_mm
  , pressao_mb
  , umidade_relativa
  , temperatura_k
  , vento_ms
	from
		resumo_clima
                    """, database.engine )

In [ ]:
for column in voos.columns:
    if column.startswith( "dt" ):
        voos[ column ] = pd.to_datetime( voos[ column ], infer_datetime_format = True )
for column in clima.columns:
    if column.startswith( "dt" ):
        clima[ column ] = pd.to_datetime( clima[ column ], infer_datetime_format = True )
voos = voos.query( "atrasado > -60" )
voos[ "atrasado" ] = voos[ "atrasado" ] >= 20

In [ ]:
df = voos.merge( clima, how = "inner",
                 left_on = [ "municipio_origem", "dt_referencia" ],
                 right_on = [ "nm_municipio", "dt_referencia" ] )
df[ "dia_semana" ] = df[ "dt_referencia" ].dt.dayofweek
df[ "semana_ano" ] = df[ "dt_referencia" ].dt.weekofyear
numericas = [ "precipitacao_mm", "pressao_mb", "umidade_relativa", "temperatura_k", "vento_ms" ]
categoricas = [ "sg_empresa_icao",
                "municipio_origem",
                "municipio_destino",
                "dia_semana",
                "semana_ano" ]
columns_keep = [ ]
columns_keep.extend( numericas )
columns_keep.extend( categoricas )
columns_keep.append( "atrasado" )
columns_keep.append( "dt_referencia" )
df = df[ columns_keep ]


In [ ]:
df['atrasado'] = df['atrasado'].astype(int)

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(df[ numericas + ['atrasado'] ], alpha = 0.2, figsize = ( 12, 12 ), diagonal = 'kde' )

A scatter matrix não mostra nenhuma correlação muito forte entre as variáveis.

In [ ]:
import seaborn as sns

sns.set(rc={'figure.figsize':(11.7,8.27)})

corr = df.corr()
ax = sns.heatmap(
    corr,
    vmin = -1,
    vmax = 1,
    center = 0,
    cmap = sns.diverging_palette( 20, 220, n = 200 ),
    square = True
)

ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation = 45,
    horizontalalignment = 'right'
)


O heatmap mostra que há pouca correlação grande maioria das variáveis, inclusive havendo correlações negativas.

In [ ]:
df2 = df.copy()

In [ ]:
df.info()

In [ ]:
df.describe()

É importante observar que os filtros definidos para simplificação do modelo remove grande parte dos dados existentes. Com mais tempo, seria interessante fazer uma análise mais detalhada e com os dados completos.

### Fazendo a limpeza dos dados NA

In [ ]:
df2.isna().sum(axis=0)/df2.shape[0]

Como aproximadamente 5% dos dados relacionados ao clima são NA, optamos por dropa-los.

In [ ]:
df.dropna(inplace=True)

## Modelo de Machine Learning

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_roc_curve, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from category_encoders import TargetEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
df_y = df[ [ "atrasado" ] ]
df_x = df[ columns_keep ].drop( [ "atrasado", "dt_referencia" ], axis = 1 )
(df_train_x, df_test_x,
 df_train_y, df_test_y) = train_test_split( df_x, df_y,
                                            train_size = 0.8,
                                            stratify = df_y )
encoder = TargetEncoder( cols = categoricas,
                         handle_unknown = 'value' ).fit( df_train_x, df_train_y )
df_train_x = encoder.transform( df_train_x )
df_test_x = encoder.transform( df_test_x )

### Inicialmente optamos por treinar um modelo de Regressão Logística e um Floresta Aleatória como base.

In [ ]:
forest: RandomForestClassifier = RandomForestClassifier( max_depth = 4, random_state = 1, )
forest: RandomForestClassifier = forest.fit( df_train_x, df_train_y )
df_train_yhat_forest = forest.predict_proba( df_train_x )[ :, [ 1 ] ]
df_test_yhat_forest = forest.predict_proba( df_test_x )[ :, [ 1 ] ]
logistic: LogisticRegression = LogisticRegression()
logistic = logistic.fit( df_train_x, df_train_y )
df_train_yhat_logistic = logistic.predict_proba( df_train_x )[ :, [ 1 ] ]
df_test_yhat_logistic = logistic.predict_proba( df_test_x )[ :, [ 1 ] ]

## Métricas do Modelo

In [ ]:
for train, test in ((df_train_yhat_forest, df_test_yhat_forest),
                    (df_train_yhat_logistic, df_test_yhat_logistic)):
    threshold = np.percentile( train, 50 )
    print( threshold )
    y_true = df_test_y
    y_pred = test >= threshold
    normalize = None
    cm = confusion_matrix( y_true,
                           y_pred, normalize = normalize,
                           )
    predictions = ConfusionMatrixDisplay.from_predictions( y_true = y_true, y_pred = y_pred,
                                                           normalize = normalize, cmap = 'Blues' )
    plt.show()

### Métricas com cross_val_predict para o Random Forest

In [ ]:
from sklearn.model_selection import cross_val_predict

df_train_pred_y = cross_val_predict( forest2, df_train_x, df_train_y, cv = 5 )

In [ ]:
confusion_matrix( df_train_y, df_train_pred_y )

In [ ]:
confusion_matrix( df_test_y, df_test_yhat_forest >= 0.5 )

In [ ]:
from pprint import pprint
pprint(confusion_matrix( df_train_y, df_train_pred_y , normalize='true'))
pprint(confusion_matrix( df_test_y, df_test_yhat_forest >= 0.5 , normalize='true'))

É possível ver que os dados de treino e teste apresentam resultados bastante semelhantes, o que é um bom sinal.

In [ ]:
predictions = ConfusionMatrixDisplay.from_predictions( y_true = df_train_y, y_pred = df_train_pred_y, normalize = normalize, cmap = 'Blues' )
plt.show()

In [ ]:
conf_norm = confusion_matrix( df_train_y, df_train_pred_y , normalize='true')

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
print(f'Precision: {precision_score(df_train_y, df_train_pred_y)}')
print(f'Recall (Sensitivity): {recall_score(df_train_y, df_train_pred_y)}')
print(f'F1 Score: {f1_score(df_train_y, df_train_pred_y)}')
print(f'Specificity: {conf_norm[0,0]/(conf_norm[0,0]+conf_norm[0,1])}')

* O Recall mostra que aproximadamente 71% dos voos que atrasaram foram corretamente identificados pelo modelo.
* A Specificity mostra que aproxidamente 82% dos voos que não atrasaram foram corretamente identificados pelo modelo.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report( df_train_y, df_train_pred_y ))

In [ ]:
fig, ax = plt.subplots()
ax.plot( [ 0, 1 ], [ 0, 1 ] )
plot_roc_curve( forest, df_train_x, df_train_y, ax = ax )
plot_roc_curve( logistic, df_train_x, df_train_y, ax = ax )

Nestas condições, é possível observar que a performance do modelo Random Forest é superior à do Logistic Regression. O AUC é extremamente útil para esta comparação.
Além disso talvez fosse interessante considerar a Precision ao invés de False Positive Rate caso os dados fossem muito desbalanceados, o que é uma estratégia interessante pelo fato de o Precision não levar em conta os true negatives. No nosso caso cerca de 25% dos voos atrasam, então talvez não tenha muita diferença.

# Ajuste fino do modelo Random Forest

### Parâmetros usados atualmente:

In [ ]:
print('Parâmetros usados atualmente:\n')
pprint(forest.get_params())

### Randomized Hyperparameter Search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [ int(x) for x in np.linspace( start = 200, stop = 2000, num = 10 ) ]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

### Treinando o modelo com Random Search

In [ ]:
forest_random = RandomizedSearchCV( estimator = forest,
                                    param_distributions = random_grid,
                                    n_iter = 100,
                                    cv = 5,
                                    verbose = 2,
                                    random_state = 1,
                                    n_jobs = -1 )

forest_random.fit( df_train_x, df_train_y )

In [ ]:
forest_random.best_params_

### Comparação entre o modelo base e o modelo com os hyper parâmetros ajustados

In [ ]:
def avalia_modelo(modelo, x_test, y_test):
    y_pred = modelo.predict(x_test)
    print(f'Precision: {precision_score(y_test, y_pred)}')
    print(f'Recall (Sensitivity): {recall_score(y_test, y_pred)}')
    print(f'F1 Score: {f1_score(y_test, y_pred)}')
    print(f"Specificity: {confusion_matrix(y_test, y_pred, normalize='true')[0,0]/(confusion_matrix(y_test, y_pred, normalize='true')[0,0]+confusion_matrix(y_test, y_pred, normalize='true')[0,1])}")
    print(f'AUC: {roc_auc_score(y_test, y_pred)}')
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'ROC: {roc_curve(y_test, y_pred)}')
    print(f'Confusion Matrix:\n{confusion_matrix(y_test, y_pred)}')
    print(f'Classification Report:\n{classification_report(y_test, y_pred)}')
    print(f'\n')

### Modelo base

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve

avalia_modelo(forest, df_test_x, df_test_y)

### Modelo com ajuste de hyper parâmetros

In [ ]:
avalia_modelo(forest_random.best_estimator_, df_test_x, df_test_y)

In [ ]:
fig, ax = plt.subplots()
ax.plot( [ 0, 1 ], [ 0, 1 ] )
plot_roc_curve( forest_random.best_estimator_, df_test_x, df_test_y, ax = ax, name='Random Forest Tunado' )
plot_roc_curve( forest, df_test_x, df_test_y, ax = ax, name='Random Forest Base' )
plot_roc_curve( logistic, df_test_x, df_test_y, ax = ax, name='Logistic Regression' )

É possível notar que o modelo com ajuste de hyper parâmetros tem uma performance melhor que o modelo base, com um acréscimo de quase 5% na maiora das métricas.

In [ ]:
forest_random.best_estimator_.feature_importances_
for feature, score in zip(df_train_x.columns, forest_random.best_estimator_.feature_importances_):
    print(f'{feature}: {score}')

Para finalizar, a semana do ano é a feature mais importante do modelo com ajuste de hyper parâmetros. O que faz todo o sentido uma vez que está diretamente relacionada à datas de feriados e férias e  consequentemente à movimentação nos aeroportos.